<h1 align="center">3.4 Корреляция и корреляционный анализ</h1>

Возьмите датасет Mortality and Water Hardness
https://www.kaggle.com/ukveteran/mortality-and-water-hardness

В этом датасете содержатся данные по средней годовой смертности на 100000 населения и концентрации кальция в питьевой воде для 61 большого города в Англии и Уэльсе. Города дополнительно поделены на северные и южные.

1. Задача - ответить на вопрос есть ли связь между жёсткостью воды и средней годовой смертностью?
- Построить точечный график
- Рассчитать коэффициенты корреляции Пирсона и Спирмена
- Построить модель линейной регрессии
- Рассчитать коэффициент детерминации
- Вывести график остатков
2. Сохраняется ли аналогичная зависимость для северных и южных городов по отдельности?
- Разделить данные на 2 группы
- Повторить аналогичные шаги из пункта 1 для каждой группы по отдельности

In [2]:
import pandas as pd
import numpy as np
import cufflinks
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import iplot, init_notebook_mode
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [3]:
df = pd.read_csv('water.csv')
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  61 non-null     int64 
 1   location    61 non-null     object
 2   town        61 non-null     object
 3   mortality   61 non-null     int64 
 4   hardness    61 non-null     int64 
dtypes: int64(3), object(2)
memory usage: 2.5+ KB


,Unnamed: 0,location,town,mortality,hardness
0,1,South,Bath,1247,105
1,2,North,Birkenhead,1668,17
2,3,South,Birmingham,1466,5
3,4,North,Blackburn,1800,14
4,5,North,Blackpool,1609,18


<h3 align="center">Взаимосвязь между смертностью и жёсткостью воды обратная</h3>

Гипотеза - cмертность меньше там, где вода мягче.

In [4]:
df.iplot(kind = 'scatter', x='hardness', y='mortality', xTitle='Hardness', yTitle='Mortality', mode='markers', 
         title={'text': 'The relationship between mortality and water hardness', 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'})

In [5]:
# взаимосвязь между смертностью и жёсткостью воды - 2D-контурная гистограмма
fig = go.Figure(go.Histogram2dcontour(x = df['hardness'], y = df['mortality'], colorscale = 'Plasma'))
fig = px.density_contour(df, x='hardness', y='mortality')
fig.update_traces(contours_coloring='fill', contours_showlabels = True)
fig.update_layout(title={'text': 'The relationship between mortality and water hardness', 'y':0.95, 'x':0.5, 
                         'xanchor': 'center', 'yanchor': 'top'})
fig.show()

C:\Users\shali\anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:716: DeprecationWarning:

plotly.graph_objs.Histogram2dcontour is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.Histogram2dContour




<h3 align="center">Коэффициенты корреляции Пирсона и Спирмена</h3>

Коэффициент корреляции показывает:
1. Силу линейной взаимосвязи между двумя переменными,
2. Направление взаимосвязи (прямая или обратная)

Это ковариация двух переменных поделить на их дисперсии. Величина коэффициента корреляции заключена в пределах -1 <= r <= 1

Коэффициенты имеют отрицательные значения, что означает, что соотношение действительно обратное. Коэффициенты имеют значения около 0,65, что означает, что теоретически может существовать какая-то взаимосвязь.

In [6]:
corrs = df[['hardness', 'mortality']].corr()

fig = ff.create_annotated_heatmap(z=corrs.values, x=list(corrs.columns), y=list(corrs.index), 
                                  annotation_text=corrs.round(2).values, showscale=True, colorscale='plasma')
fig.show()

In [7]:
spear = df[['hardness', 'mortality']].corr(method='spearman')

fig = ff.create_annotated_heatmap(z=spear.values, x=list(spear.columns), y=list(spear.index),
    annotation_text=spear.round(2).values, showscale=True, colorscale='plasma')
fig.show()

<h3 align="center">Модель линейной регрессии</h3>

Регрессионный анализ – инструмент для количественного предсказания значения одной переменной на основании другой. Регрессия – предсказание одной переменной на основании другой. Одна переменная – независимая, а другая – зависимая.

Регрессионный анализ

По количеству независимых переменных:
- простой (регрессия между двумя переменными);
- множественной (регрессия между зависимой переменной Y и
несколькими независимыми переменными (X1, X2, …, Xn)).

По типу зависимости:
- линейный
- нелинейный

Общий подход к решению
1. Определение формы зависимости
2. Построение модели регрессии
3. Оценка неизвестных значений зависимой переменной

Построение модели регрессии
1. Мы выбрали форму регрессии. Предположим, это прямая линия y=ax+b
2. У нас есть выборка точек с измеренными значениями y и x
3. Нужно подобрать наилучшие параметры a, b, которые максимально точно описывают наши данные.

In [8]:
x = df[['hardness']]
y = df['mortality']

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)

In [10]:
lr = np.array(df.hardness)[:, None]

model = LinearRegression()
model.fit(x_train.values, y_train)

x_range = np.linspace(lr.min(), lr.max(), 100)
y_range = model.predict(x_range.reshape(-1, 1))

fig = go.Figure([
    go.Scatter(x=x_train.squeeze(), y=y_train, name='train', mode='markers'),
    go.Scatter(x=x_test.squeeze(), y=y_test, name='test', mode='markers'),
    go.Scatter(x=x_range, y=y_range, name='prediction'),
])

fig.update_layout(title={'text': 'The relationship between mortality and water hardness', 'y':0.9, 'x':0.5, 
                         'xanchor': 'center', 'yanchor': 'top'}, xaxis_title='Hardness', yaxis_title='Mortality')
fig.show()

In [11]:
print(model.coef_)
print(model.intercept_)

[-3.13167087]
1668.9722977277095


In [12]:
y_pred = model.predict(x_test.values).round(2)
y_pred

array([1340.15, 1637.66, 1625.13, 1531.18, 1424.7 , 1606.34, 1252.46,
       1434.1 , 1603.21, 1650.18, 1484.2 , 1415.31, 1546.84, 1625.13,
       1446.62, 1546.84, 1637.66, 1622.  , 1352.67])

<h3 align="center">Коэффициент детерминации</h3>

Коэффициент детерминации - это доля дисперсии зависимой переменной, объясняемая рассматриваемой моделью зависимости, то есть объясняющими переменными.

In [13]:
model.score(x_test.values, y_test).round(3)

0.505

Коэффициент детерминации мал.

<h3 align="center">График остатков</h3>

Распределение остатков отличается от нормального, однако тестовые остатки меньше, чем при обучении. Согласно этой модели, невозможно однозначно сказать, зависит ли смертность от жесткости воды или нет. Необходимо провести дополнительные исследования.

In [14]:
x2_train, x2_test = train_test_split(df.index, test_size=.25, random_state=0)
df['split'] = 'train'
df.loc[x2_test, 'split'] = 'test'

x_2 = df[['hardness']]
x_2_train = df.loc[x2_train, ['hardness']]
y2_train = df.loc[x2_train, 'mortality']

model = LinearRegression()
model.fit(x_2_train, y2_train)
df['prediction'] = model.predict(x_2)
df['residual'] = df['prediction'] - df['mortality']

fig = px.scatter(df, x='prediction', y='residual', marginal_y='violin', color='split', trendline='ols')

fig.update_layout(title={'text': 'Residual plot', 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'})
fig.show()

<h3 align="center">Взаимосвязь для северных городов</h3>

Коэффициент корреляции для северных городов - Пирсона -0.37, а Спирмена -0.404. Это говорит о том, что взаимосвязи почти нет.

In [15]:
df_north = df[df['location'] == 'North']
df_south = df[df['location'] == 'South']

In [16]:
df_north.iplot(kind = 'scatter', x='hardness', y='mortality', xTitle='Hardness', yTitle='Mortality', mode='markers',  
         title={'text': 'The relationship between mortality and water hardness in northern cities', 
                'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'});

In [17]:
fig = go.Figure(go.Histogram2dcontour(x = df_north['hardness'], y = df_north['mortality'], 
                                      colorscale = 'Plasma'))
fig = px.density_contour(df_north, x='hardness', y='mortality')
fig.update_traces(contours_coloring='fill', contours_showlabels = True)
fig.update_layout(title={
    'text': 'The relationship between mortality and water hardness in northern cities', 
    'y':0.95, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'})
fig.show()

C:\Users\shali\anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:716: DeprecationWarning:

plotly.graph_objs.Histogram2dcontour is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.Histogram2dContour




In [18]:
corrs_north = df_north[['hardness', 'mortality']].corr()

fig = ff.create_annotated_heatmap(z=corrs_north.values, x=list(corrs_north.columns), 
                                  y=list(corrs_north.index), 
                                  annotation_text=corrs_north.round(2).values, 
                                  showscale=True, colorscale='plasma')
fig.show()

In [19]:
spear_north = df_north[['hardness', 'mortality']].corr(method='spearman')

fig = ff.create_annotated_heatmap(z=spear_north.values, x=list(spear_north.columns), 
                                  y=list(spear_north.index), 
                                  annotation_text=spear_north.round(3).values, showscale=True, colorscale='plasma')
fig.show()

In [20]:
x_north = df_north[['hardness']]
y_north = df_north['mortality']

In [21]:
x_train_north, x_test_north, y_train_north, y_test_north = train_test_split(x_north, y_north, test_size=0.30, random_state=42)

In [22]:
lr_north = np.array(df_north.hardness)[:, None]

model_north = LinearRegression()
model_north.fit(x_train_north.values, y_train_north)

x_range_north = np.linspace(lr_north.min(), lr_north.max(), 100)
y_range_north = model_north.predict(x_range_north.reshape(-1, 1))

fig = go.Figure([
    go.Scatter(x=x_train_north.squeeze(), y=y_train_north, name='train', mode='markers'),
    go.Scatter(x=x_test_north.squeeze(), y=y_test_north, name='test', mode='markers'),
    go.Scatter(x=x_range_north, y=y_range_north, name='prediction'),
])

fig.update_layout(title={
    'text': 'The relationship between mortality and water hardness northern cities', 
    'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, xaxis_title='Hardness', 
                  yaxis_title='Mortality')
fig.show()

In [23]:
print(model_north.coef_)
print(model_north.intercept_)

[-1.97686226]
1688.150229031281


In [24]:
y_pred_north = model_north.predict(x_test_north.values).round(2)
y_pred_north

array([1672.34, 1670.36, 1660.47, 1676.29, 1656.52, 1547.79, 1601.17,
       1672.34, 1611.05, 1658.5 , 1611.05])

In [25]:
model_north.score(x_test_north.values, y_test_north).round(3)

0.019

In [26]:
x2_train_north, x2_test_north = train_test_split(df_north.index, test_size=.25, random_state=0)
df_north['split'] = 'train'
df_north.loc[x2_test_north, 'split'] = 'test'

x_2_north = df_north[['hardness']]
x_2_train_north = df_north.loc[x2_train_north, ['hardness']]
y2_train_north = df_north.loc[x2_train_north, 'mortality']

model_north_2 = LinearRegression()
model_north_2.fit(x_2_train_north, y2_train_north)
df_north['prediction'] = model_north_2.predict(x_2_north)
df_north['residual'] = df_north['prediction'] - df_north['mortality']

fig = px.scatter(df_north, x='prediction', y='residual', marginal_y='violin', color='split', 
                 trendline='ols')

fig.update_layout(title={'text': 'Residual plot north', 'y':0.95, 'x':0.5, 
                         'xanchor': 'center', 'yanchor': 'top'})
fig.show()

<h3 align="center">Взаимосвязь для южных городов</h3>

In [ ]:
df_south.iplot(kind = 'scatter', x='hardness', y='mortality', xTitle='Hardness', yTitle='Mortality', 
               mode='markers',  
               title={'text': 'The relationship between mortality and water hardness in southern cities', 
                'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'});

In [ ]:
fig = go.Figure(go.Histogram2dcontour(x = df_south['hardness'], y = df_south['mortality'], 
                                      colorscale = 'Plasma'))
fig = px.density_contour(df_south, x='hardness', y='mortality')
fig.update_traces(contours_coloring='fill', contours_showlabels = True)
fig.update_layout(title={
    'text': 'The relationship between mortality and water hardness in southern cities', 
    'y':0.95, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'})
fig.show()

In [ ]:
corrs_south = df_south[['hardness', 'mortality']].corr()

fig = ff.create_annotated_heatmap(z=corrs_south.values, x=list(corrs_south.columns), 
                                  y=list(corrs_south.index), 
                                  annotation_text=corrs_south.round(2).values, showscale=True, 
                                  colorscale='plasma')
fig.show()

In [ ]:
spear_south = df_south[['hardness', 'mortality']].corr(method='spearman')

fig = ff.create_annotated_heatmap(z=spear_south.values, x=list(spear_south.columns), 
                                  y=list(spear_south.index), 
                                  annotation_text=spear_south.round(2).values, showscale=True, 
                                  colorscale='plasma')
fig.show()

In [ ]:
x_south = df_south[['hardness']]
y_south = df_south['mortality']

In [ ]:
x_train_south, x_test_south, y_train_south, y_test_south = train_test_split(x_south, y_south, test_size=0.30, random_state=42)

In [ ]:
lr_south = np.array(df_south.hardness)[:, None]

model_south = LinearRegression()
model_south.fit(x_train_south.values, y_train_south)

x_range_south = np.linspace(lr_south.min(), lr_south.max(), 100)
y_range_south = model_south.predict(x_range_south.reshape(-1, 1))

fig = go.Figure([
    go.Scatter(x=x_train_south.squeeze(), y=y_train_south, name='train', mode='markers'),
    go.Scatter(x=x_test_south.squeeze(), y=y_test_south, name='test', mode='markers'),
    go.Scatter(x=x_range_south, y=y_range_south, name='prediction'),
])

fig.update_layout(title={
    'text': 'The relationship between mortality and water hardness southern cities', 
    'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  xaxis_title='Hardness', yaxis_title='Mortality')
fig.show()


In [ ]:
print(model_south.coef_)
print(model_south.intercept_)

In [ ]:
y_pred_south = model_south.predict(x_test_south.values).round(2)
y_pred_south

In [ ]:
model_south.score(x_test_south.values, y_test_south).round(3)

In [ ]:
x2_train_south, x2_test_south = train_test_split(df_south.index, test_size=.25, random_state=0)
df_south['split'] = 'train'
df_south.loc[x2_test_south, 'split'] = 'test'

x_2_south = df_south[['hardness']]
x_2_train_south = df_south.loc[x2_train_south, ['hardness']]
y2_train_south = df_south.loc[x2_train_south, 'mortality']

model_south_2 = LinearRegression()
model_south_2.fit(x_2_train_south, y2_train_south)
df_south['prediction'] = model_south_2.predict(x_2_south)
df_south['residual'] = df_south['prediction'] - df_south['mortality']

fig = px.scatter(df_south, x='prediction', y='residual', marginal_y='violin', color='split', 
                 trendline='ols')

fig.update_layout(title={'text': 'Residual plot south', 'y':0.95, 'x':0.5, 
                         'xanchor': 'center', 'yanchor': 'top'})
fig.show()

Коэффициент детерминации, рассчитанный для городов южного региона, имеет отрицательное значение. Разработанная модель дает прогноз хуже, чем просто среднее значение. Данная модель не подходит для практического использования.

## Выводы
1. Теоретически, располагая более полными данными, можно предположить, что существует какая-то взаимосвязь между жесткостью воды и смертностью. На это указывает первая модель линейной регрессии.
2. Для городов на севере эта взаимосвязь практически отсутствует.
3. Для городов на юге нет никакой взаимосвязи.

![](Plotly_logo.png)